In [ ]:
from enum import Enum
import json
from optimade.adapters import Structure
from optimade_gateway.models import DatabasesResponse, QueriesResponseSingle
from pydantic import ValidationError
from random import randint
import requests
from time import sleep

OPTIMADE_GATEWAY_BASE_URL = "https://optimade-gateway.herokuapp.com"

Let's check what databases are available.

In [ ]:
response = requests.get(f"{OPTIMADE_GATEWAY_BASE_URL}/databases")
if response.ok:
    databases_response = DatabasesResponse(**response.json())
else:
    raise RuntimeError(f"Unsuccessful response.\nStatus code: {response.status_code}\nResponse:\n{response.json()}")

print("Available databases:")
print("\n".join(f"- {_.id} ({_.attributes.base_url})" for _ in databases_response.data))

Use the database ID (e.g., `mcloud/sssp`) in the list of `database_ids` when using the `/search` endpoint.
Otherwise, simply use the [standard OPTIMADE `/structures` query parameters](https://github.com/Materials-Consortia/OPTIMADE/blob/develop/optimade.rst#entry-listing-url-query-parameters).

In [ ]:
database_ids = [
    "mcloud/sssp",
    "cod",
    "mp",
    "mcloud/mc3d-structures",
]

optimade_filter = 'elements HAS ALL "Si","Al","Mg"'

In [ ]:
response = requests.get(
    f"{OPTIMADE_GATEWAY_BASE_URL}/search",
    params={
        "database_ids": database_ids,
        "filter": optimade_filter,
        "timeout": 30,
    }
)
if response.ok:
    search_response = QueriesResponseSingle(**response.json())
else:
    raise RuntimeError(f"Unsuccessful response.\nStatus code: {response.status_code}\nResponse:\n{json.dumps(response.json(), indent=2)}")

print("Query overview:")
print("\n".join(f"- {id_} (Returned results: {len(data)})" for id_, data in search_response.data.attributes.response.data.items()))

In [ ]:
structures = {}
for database, database_structures in search_response.data.attributes.response.data.items():
    structures[database] = []
    for structure in database_structures:
        try:
            structures[database].append(Structure(structure.dict()))
        except ValidationError:
            # print(f"Skipping structure (id={structure.id}) from database {database}")
            continue

print("\nQuery overview (after validating structure properties):")
print("\n".join(f"- {id_} (Valid results: {len(data)})" for id_, data in structures.items()))

The `structures` dictionary now holds all the structures that validate against the [OPTIMADE Python tools](https://github.com/Materials-Consortia/optimade-python-tools) structure-type model.
The keys of the `structures` dictionary are the database IDs, while the values are a list of [OPTIMADE Python tools `Structure` adapters](https://www.optimade.org/optimade-python-tools/latest/api_reference/adapters/structures/adapter/#optimade.adapters.structures.adapter.Structure), with which one can convert the structure to other well-known formats, e.g., ASE `Atoms`, AiiDA `StructureData` or even a CIF file or PDB file.
For the latter, it will return the string content for such a file, you will have to write it to a file object yourself.

The following is a showcase of converting a random structure in `structures` to a CIF file syntax:

In [ ]:
for db_structures in structures.values():
    if db_structures:
        a_structure = db_structures[randint(0, len(db_structures)-1)]
print(a_structure.as_ase)

### MarketPlace

Using the OPTIMADE Gateway from the MarketPlace.

Changes:
- Go through the proxy service, using capabilities.
- Provide an authorization token (if you want to identify yourself to the OPTIMADE servers).

In [ ]:
MARKETPLACE_OPTIMADE_APP = "https://staging.the-marketplace.eu/api/proxy/proxy/1bb6b264-d896-47e2-b54d-6d49e05299f2"

class Capabilities(Enum):
    """Implemented capabilities by the OPTIMADE Gateway."""

    HEARTBEAT = "heartbeat"  # GET /heartbeat
    GET_COLLECTION = "getCollection"  # GET /queries/<ID>
    QUERY_COLLECTION = "queryCollection"  # GET /search
    POST_QUERY_COLLECTION = "postQueryCollection"  # POST /search

Now:

1. Follow [this guide](https://marketplace.pages.fraunhofer.de/platform/documentation/application-provider-docs/access_token.html) to obtain an access token.
1. Paste in the token in the code cell below.
1. "Purchase" the "OPTIMADE Gateway" application from [the MarketPlace App Store](https://staging.the-marketplace.eu/appstore).

In [ ]:
# Paste in your authentication token between the quotations (`"`) here:
USER_AUTH_TOKEN = ""

response = requests.get(f"{MARKETPLACE_OPTIMADE_APP}/{Capabilities.HEARTBEAT.value}", headers={"Authorization": f"Bearer {USER_AUTH_TOKEN}"})
if response.ok:
    print(response.text)
else:
    raise RuntimeError(f"Response:\n{response.text}")

In [ ]:
response = requests.post(
    f"{MARKETPLACE_OPTIMADE_APP}/{Capabilities.POST_QUERY_COLLECTION.value}",  # POST /search
    json={
        "database_ids": database_ids,
        "query_parameters": {"filter": optimade_filter},
    },
    headers={"Authorization": f"Bearer {USER_AUTH_TOKEN}"},
)
if response.ok:
    search_response = QueriesResponseSingle(**response.json())
else:
    try:
        parsed_response = json.dumps(response.json(), indent=2)
        if "_gateway_traceback" in response.json().get("meta", {}):
            parsed_response += f"\nTraceback:\n{response.json()['meta']['_gateway_traceback']}"
    except json.JSONDecodeError:
        parsed_response = response.text
    raise RuntimeError(f"Unsuccessful response.\nStatus code: {response.status_code}\nResponse:\n{parsed_response}")

state = search_response.data.attributes.state
while state != state.FINISHED:
    sleep(2)
    response = requests.get(
        f"{MARKETPLACE_OPTIMADE_APP}/{Capabilities.GET_COLLECTION.value}",  # GET /queries/<ID>
        params={"query_id": search_response.data.id},
        headers={"Authorization": f"Bearer {USER_AUTH_TOKEN}"},
    )
    if response.ok:
        query_response = QueriesResponseSingle(**response.json())
        state = query_response.data.attributes.state
    else:
        try:
            parsed_response = json.dumps(response.json(), indent=2)
            if "_gateway_traceback" in response.json().get("meta", {}):
                parsed_response += f"\nTraceback:\n{response.json()['meta']['_gateway_traceback']}"
        except json.JSONDecodeError:
            parsed_response = response.text
        raise RuntimeError(f"Unsuccessful response.\nStatus code: {response.status_code}\nResponse:\n{parsed_response}")

print("Query overview:")
print("\n".join(f"- {id_} (Returned results: {len(data)})" for id_, data in query_response.data.attributes.response.data.items()))

In [ ]:
structures = {}
for database, database_structures in query_response.data.attributes.response.data.items():
    structures[database] = []
    for structure in database_structures:
        try:
            structures[database].append(Structure(structure.dict()))
        except ValidationError:
            continue
            # print(f"Skipping structure (id={structure.id}) from database {database}")

print("\nQuery overview (after validating structure properties):")
print("\n".join(f"- {id_} (Valid results: {len(data)})" for id_, data in structures.items()))

In [ ]:
for db_structures in structures.values():
    if db_structures:
        a_structure = db_structures[randint(0, len(db_structures)-1)]
print(a_structure.as_pymatgen)